# ResNet_implementation

Tensorflow를 이용해서 ReNet을 구현해 볼 것이다. 

출처 : https://blog.naver.com/et2035/221371125386

In [163]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # warning 출력 방지
from keras.applications.resnet50 import ResNet50, decode_predictions
resnet = ResNet50()

다음은 ResNet50의 구조이다.

In [96]:
resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

### CPU로 학습하기 때문에 ImageNet, CIFAR-10을 사용하진 않고, 네트워크만을 이해하기 위해 MNIST에 적용하여 이해할 것 이다. 

### 그리고 ResNet50을 이용하지 않고 논문에 나온 그 network만을 구현할 것이다. 

In [4]:
import tqdm
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels

In [6]:
y_train[:3]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [287]:
# image preprocessing
X_train = train.values.astype('float32')/255
X_test = train.values.astype('float32')/255

In [296]:
# reshape
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [297]:
X = tf.placeholder(tf.float32, [None,28,28,1])
y = tf.placeholder(tf.float32,[None,10])

In [298]:
def resnet(input):
    res = tf.nn.conv2d(input,tf.Variable(tf.random_normal([3,3,32,32]))
                       , strides=[1,1,1,1], padding='SAME')
    res = tf.nn.relu(res)
    res = tf.nn.conv2d(input,tf.Variable(tf.random_normal([3,3,32,32]))
                       , strides=[1,1,1,1], padding='SAME')
    res = input + res
    res = tf.nn.relu(res)
    return res

In [299]:
layer = tf.nn.conv2d(X,tf.Variable(tf.random_normal([3,3,1,32]))
                     ,strides=[1,1,1,1],padding='SAME')

for i in range(5):
    layer = resnet(layer)

In [300]:
final = tf.nn.conv2d(layer,tf.Variable(tf.random_normal([3,3,32,1]))
                       , strides=[1,1,1,1], padding='SAME')
# 쭉 펴준다. (28*28 = 784)   
flatten = tf.reshape(final,[-1,final.get_shape()[1:4].num_elements()])
W = tf.Variable(tf.random_normal([final.get_shape()[1:4].num_elements(),10]))
result = tf.matmul(flatten,W)

In [301]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=result,labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(result,1),tf.argmax(y,1)),dtype =  tf.float32))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [302]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [306]:
batch_size=50

cost_list=[]
acc_list=[]
for i in range(100):
    for j in tqdm.tqdm_notebook(range(mnist.train.num_examples//batch_size)):
        _,c = sess.run([optimizer,cost],feed_dict={X:X_train[j*batch_size:(j+1)*batch_size,:,:,:],y:y_train[j*batch_size:(j+1)*batch_size,:]})
        cost_list.append(c)
    print("Epoch:{0}, Cost:{1}".format(i+1, np.mean(cost_list)))
    print("Accuracy :",sess.run(accuracy, feed_dict={X:X_test,y:y_test}))


Epoch:1, Cost:1958.700927734375
Accuracy : 0.9062


Epoch:2, Cost:1507.1817626953125
Accuracy : 0.8986


Epoch:3, Cost:1207.7093505859375
Accuracy : 0.9129


Epoch:4, Cost:987.8175048828125
Accuracy : 0.9225


Epoch:5, Cost:828.54638671875
Accuracy : 0.9228


...

Epoch:8, Cost:546.6304321289062
Accuracy : 0.8969


Epoch:9, Cost:488.52044677734375
Accuracy : 0.9159


Epoch:10, Cost:440.9261474609375
Accuracy : 0.9161


Epoch:11, Cost:401.5045166015625


KeyboardInterrupt: 

In [8]:
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])